In [ ]:
import os
import json
import pandas as pd
from energyemissionsregio.config import DATA_PATH
import matplotlib.pyplot as plt
import seaborn as sns

### potential predictor variables

In [ ]:
lau_vars = [
    "population", 
             "area", 
             'continuous_urban_fabric_cover',
       'discontinuous_urban_fabric_cover',
       'industrial_or_commercial_units_cover',
       'port_areas_cover',
       'airports_cover', 'mineral_extraction_sites_cover',
       'dump_sites_cover', 'construction_sites_cover',
       'green_urban_areas_cover', 'sport_and_leisure_facilities_cover',
       'non_irrigated_arable_land_cover',
       'permanently_irrigated_land_cover', 'rice_fields_cover',
       'vineyards_cover', 'fruit_trees_and_berry_plantations_cover',
       'olive_groves_cover', 'pastures_cover', 'permanent_crops_cover',
       'complex_cultivation_patterns_cover',
       'agriculture_with_natural_vegetation_cover',
       'agro_forestry_areas_cover', 'broad_leaved_forest_cover',
       'coniferous_forest_cover', 'mixed_forest_cover',
       'natural_grasslands_cover', 'moors_and_heathland_cover',
       'sclerophyllous_vegetation_cover',
       'transitional_woodland_shrub_cover',
       'beaches_dunes_and_sand_cover', 
       'bare_rocks_cover',
       'sparsely_vegetated_areas_cover', 'burnt_areas_cover',
       'glaciers_and_perpetual_snow_cover', 'inland_marshes_cover',
       'peat_bogs_cover', 
       'salt_marshes_cover', 
       'salines_cover',
       'intertidal_flats_cover', 'water_courses_cover',
       'water_bodies_cover', 'coastal_lagoons_cover', 'estuaries_cover',
       'sea_and_ocean_cover',
       "number_of_iron_and_steel_industries",
      "number_of_cement_industries",
      "number_of_refineries",
      "number_of_paper_and_printing_industries",
      "number_of_chemical_industries",
      "number_of_glass_industries",
      "number_of_non_ferrous_metals_industries",
      "number_of_non_metallic_minerals_industries",
       "railway_network",
       "road_network",
       "number_of_buildings",
       'average_air_pollution_due_to_pm2.5',
       'average_air_pollution_due_to_no2',
       'average_air_pollution_due_to_o3',
       'average_air_pollution_due_to_pm10',
       'number_of_buffaloes', 'number_of_cattle', 'number_of_pigs',
       'number_of_sheeps', 'number_of_chickens', 'number_of_goats',
             ]


In [ ]:
# NUTS3 data with no missing values
nuts3_vars = ["employment_in_agriculture_forestry_and_fishing", 
                  "employment_in_manufacturing",
                  "employment_in_construction",
                  "gross_domestic_product",
                  "road_transport_of_freight",
                  "soil_sealing",
                  "cproj_annual_mean_temperature_heating_degree_days",
                  "cproj_annual_mean_temperature_cooling_degree_days",
             ]


In [ ]:
var_df = pd.read_excel(
    os.path.join(DATA_PATH, "..", "..", "01_raw", "variables_with_details_and_tags.xlsx"),
    sheet_name="collected_variables_EU",
)

In [ ]:
x_vars_lau = lau_vars.copy()
x_vars_nuts3 = nuts3_vars.copy()

In [ ]:
X_vars_df = None

for var_name in x_vars_lau:
    
    _df = pd.read_csv(
        os.path.join(DATA_PATH, f"{var_name}.csv")
    )
    _df = _df[_df["region_code"].str.startswith("DE")][["region_code", "value"]].copy()

    _df = _df.fillna(0) # filling NAs for point vars. Non-point vars have no NAs in Germany

    #convert LAU to NUTS3 regions
    _df["region_code"] = _df["region_code"].str.split("_").str[0]

    # aggregate per NUTS3 region 
    agg_method = var_df[var_df["var_name"] == var_name][
            "var_aggregation_method"
        ].values[0]

    if agg_method == "SUM":
        _df = _df.groupby("region_code").sum().reset_index()
    elif agg_method == "AVG":
        _df = _df.groupby("region_code").mean().reset_index()
    elif agg_method == "MAX":
        _df = _df.groupby("region_code").max().reset_index()
    else:
        raise ValueError("Unknown var aggregation method")

    _df.rename(columns={"value": var_name}, inplace=True)

    if X_vars_df is not None:
        X_vars_df = pd.merge(X_vars_df, _df, on="region_code", how="outer")
    else:
        X_vars_df = _df

In [ ]:
for var_name in x_vars_nuts3:
    if var_name.startswith("cproj_"):
        _df = pd.read_csv(os.path.join(DATA_PATH, "..", "climate_projections", "DE", var_name, "2020.csv"))
        _df = _df[_df["climate_experiment"] == "RCP4.5"].copy()

        _df.drop(columns="climate_experiment", inplace=True)
        
    else:
        _df = pd.read_csv(
            os.path.join(DATA_PATH, f"{var_name}.csv")
        )
        _df = _df[_df["region_code"].str.startswith("DE")][["region_code", "value"]].copy()
    
    _df.rename(columns={"value": var_name}, inplace=True)

    if X_vars_df is not None:
        X_vars_df = pd.merge(X_vars_df, _df, on="region_code", how="outer")
    else:
        X_vars_df = _df

In [ ]:
X_vars_df.isna().any().any()

### drop 0  variance columns

In [ ]:

numerical_cols = X_vars_df.select_dtypes(include=['number'])
variance = numerical_cols.var()

# Identify columns with zero variance
zero_variance_cols = variance[variance == 0].index

In [ ]:
zero_variance_cols

In [ ]:
X_vars_df.drop(columns=zero_variance_cols, inplace=True)

### between paris of variables that are highly correlated, drop 1 variable

In [ ]:
relevant_vars_corr = X_vars_df.copy()
relevant_vars_corr.drop(columns="region_code", inplace=True)

corr_df = relevant_vars_corr.corr()

for idx, row in corr_df.iterrows():
    temp_dict = dict(row)
    for key, value in temp_dict.items():
        if (idx != key) & (value>=0.9):
            print(f"{idx} and {key} are highly correlated")

NOTE: some variable pairs did not show high correlation at LAU level, but when aggregation to NUTS3, they show high correlation. This shows that the strong relationships that exists at NUTS3 cannot be used at LAU to disaggregate data

In [ ]:
vars_to_drop = ["population", "bare_rocks_cover", "salt_marshes_cover", "average_air_pollution_due_to_pm10"]

In [ ]:
X_vars_df.drop(columns=vars_to_drop, inplace=True)

In [ ]:
vars_to_impute = ["de_employment_in_food_and_beverage_manufacturing",
                  "de_employment_in_mechanical_and_automotive_engineering",
                  "de_employment_in_mechatronics_energy_and_electrical",
                  "de_employment_in_agriculture",
                  "de_employment_in_wood_processing",
                  "de_employment_in_textile_and_leather_manufacturing",
                  "de_number_of_passenger_cars_emission_group_euro_1",
"de_number_of_passenger_cars_emission_group_euro_2",
"de_number_of_passenger_cars_emission_group_euro_3",
"de_number_of_passenger_cars_emission_group_euro_4",
"de_number_of_passenger_cars_emission_group_euro_5",
"de_number_of_passenger_cars_emission_group_euro_6r",
"de_number_of_passenger_cars_emission_group_euro_6dt",
"de_number_of_passenger_cars_emission_group_euro_6d",
"de_number_of_passenger_cars_emission_group_euro_other",
"de_residential_building_living_area",
"de_non_residential_building_living_area"
]

In [ ]:
for var_to_impute in vars_to_impute:
    y_var_df = pd.read_csv(
        os.path.join(DATA_PATH, f"{var_to_impute}.csv")
    )

    ## only Germany data is taken because the list of vars in vars_to_impute are avaiable only for Germany. Missing value imputation is done for the regions in Germany with missing data 
    # and all the regions in Spain
    y_var_df = y_var_df[y_var_df["region_code"].str.startswith("DE")][["region_code", "value"]].copy()
    y_var_df.rename(columns={"value": var_to_impute}, inplace=True)

    final_df_with_reg_code = pd.merge(X_vars_df, y_var_df, on="region_code", how="outer")

    final_df = final_df_with_reg_code.copy()
    final_df.drop(columns="region_code", inplace=True)

    for corr_threshold in [0.1, 0.5]:
        final_df = final_df.reindex(sorted(final_df.columns), axis=1)

        correlations = final_df.corr()[[var_to_impute]].drop(var_to_impute)
        correlations = correlations[(correlations[var_to_impute] <=-corr_threshold) | (correlations[var_to_impute] >=corr_threshold)]

        correlations = correlations.transpose()

        chosen_vars = list(correlations.columns)
        chosen_vars.extend([var_to_impute])

        save_df = final_df[chosen_vars].copy()

        save_df.to_csv(os.path.join("..", "..", "data", 
                                    "missing_value_imputation", 
                                    f"{var_to_impute}_{corr_threshold}corr.csv"), index=False)
        
        predictor_vars = list(save_df.columns)
        predictor_vars.remove(var_to_impute)

        with open(
            os.path.join("..", "..", "data", "missing_value_imputation", 
                         "predictor_vars", 
                         f"{var_to_impute}_{corr_threshold}corr.json"), "w"
        ) as fp:
            json.dump(list(predictor_vars), fp)

### correlation maps

In [ ]:
vars_to_impute = ["de_employment_in_food_and_beverage_manufacturing",
                  "de_employment_in_mechanical_and_automotive_engineering",
                  "de_employment_in_mechatronics_energy_and_electrical",
                  "de_employment_in_agriculture",
                  "de_employment_in_wood_processing",
                  "de_employment_in_textile_and_leather_manufacturing",
]

In [ ]:
corr_df = X_vars_df.copy()
for var_to_impute in vars_to_impute:
    y_var_df = pd.read_csv(
        os.path.join(DATA_PATH, f"{var_to_impute}.csv")
    )

    y_var_df = y_var_df[y_var_df["region_code"].str.startswith("DE")][["region_code", "value"]].copy()
    y_var_df.rename(columns={"value": var_to_impute}, inplace=True)

    corr_df = pd.merge(corr_df, y_var_df, on="region_code", how="outer")

corr_df.drop(columns="region_code", inplace=True)

correlations = corr_df.corr()[vars_to_impute].drop(vars_to_impute)
correlations = correlations.round(2)
correlations = correlations.transpose()

# Plotting the heatmap
plt.figure(figsize=(15, 3))  # Adjust the figure size as needed
sns.heatmap(correlations.abs(), annot=True, cmap="Blues", cbar=True, annot_kws={"rotation": 90}, vmin=0, vmax=1 )
plt.savefig(os.path.join("..", "..", "figures", "missing_value_imputation", 
                            f"employment_corr.png"), 
                            bbox_inches='tight')  # Save the figure as a PNG file

In [ ]:
vars_to_impute = [
                  "de_number_of_passenger_cars_emission_group_euro_1",
"de_number_of_passenger_cars_emission_group_euro_2",
"de_number_of_passenger_cars_emission_group_euro_3",
"de_number_of_passenger_cars_emission_group_euro_4",
"de_number_of_passenger_cars_emission_group_euro_5",
"de_number_of_passenger_cars_emission_group_euro_6r",
"de_number_of_passenger_cars_emission_group_euro_6dt",
"de_number_of_passenger_cars_emission_group_euro_6d",
"de_number_of_passenger_cars_emission_group_euro_other",
]

In [ ]:
corr_df = X_vars_df.copy()
for var_to_impute in vars_to_impute:
    y_var_df = pd.read_csv(
        os.path.join(DATA_PATH, f"{var_to_impute}.csv")
    )

    y_var_df = y_var_df[y_var_df["region_code"].str.startswith("DE")][["region_code", "value"]].copy()
    y_var_df.rename(columns={"value": var_to_impute}, inplace=True)

    corr_df = pd.merge(corr_df, y_var_df, on="region_code", how="outer")

corr_df.drop(columns="region_code", inplace=True)

correlations = corr_df.corr()[vars_to_impute].drop(vars_to_impute)
correlations = correlations.round(2)
correlations = correlations.transpose()

# Plotting the heatmap
plt.figure(figsize=(15, 5))  # Adjust the figure size as needed
sns.heatmap(correlations.abs(), annot=True, cmap="Blues", cbar=True, annot_kws={"rotation": 90}, vmin=0, vmax=1 )
plt.savefig(os.path.join("..", "..", "figures", "missing_value_imputation", 
                            f"passenger_cars_emissions.png"), 
                            bbox_inches='tight')  # Save the figure as a PNG file

In [ ]:
vars_to_impute = [
"de_residential_building_living_area",
"de_non_residential_building_living_area"
]

In [ ]:
corr_df = X_vars_df.copy()
for var_to_impute in vars_to_impute:
    y_var_df = pd.read_csv(
        os.path.join(DATA_PATH, f"{var_to_impute}.csv")
    )

    y_var_df = y_var_df[y_var_df["region_code"].str.startswith("DE")][["region_code", "value"]].copy()
    y_var_df.rename(columns={"value": var_to_impute}, inplace=True)

    corr_df = pd.merge(corr_df, y_var_df, on="region_code", how="outer")

corr_df.drop(columns="region_code", inplace=True)

correlations = corr_df.corr()[vars_to_impute].drop(vars_to_impute)
correlations = correlations.round(2)
correlations = correlations.transpose()

# Plotting the heatmap
plt.figure(figsize=(15, 1))  # Adjust the figure size as needed
sns.heatmap(correlations.abs(), annot=True, cmap="Blues", cbar=True, annot_kws={"rotation": 90}, vmin=0, vmax=1 )
plt.savefig(os.path.join("..", "..", "figures", "missing_value_imputation", 
                            f"building_living_area.png"), 
                            bbox_inches='tight')  # Save the figure as a PNG file